# Basics of Iterative Inference Programming in Gen

This tutorial introduces the basics of inference programming in Gen using **iterative inference programs**, which include **Markov chain Monte Carlo (MCMC) algorithms**.

## The task: curve-fitting with outliers

Suppose we have a dataset of points in the $x,y$ plane that is _mostly_ explained by a linear relationship, but which also has several outliers. \
We will 
1. automatically identify the outliers and 
2. find a linear relationship (a slope & intercept, as well as an inherent noise level) that explains the rest of the points:

<img src="../tutorials/images/example-inference.png" alt="See https://dspace.mit.edu/bitstream/handle/1721.1/119255/MIT-CSAIL-TR-2018-020.pdf, Figure 2(a))" width="600"/>

## Outline

**Section 1.** [Writing the model](#writing-model)

**Section 2.** [Visualizing the model's behavior](#visualizing)

**Section 3.** [ The problem with generic importance sampling ](#importance)

**Section 4.1** [MCMC Inference: Introduction](#mcmc-1)

**Section 4.2** [MCMC Inference: Block Resimulation](#mcmc-2)

**Section 4.3** [MCMC Inference: Gaussian Drift](#mcmc-3)

**Section 4.3** [MCMC Inference: Data-driven proposals based on heuristics](#mcmc-4)

In [1]:
using Gen
import Random

## 1. Writing the model <a name="writing-model"></a>

We begin, as usual, by writing a model: a Julia function responsible (conceptually) for simulating a fake dataset.

In [2]:
@gen function model(xs::Vector{Float64})
    slope = @trace(normal(0, 2), :slope)
    intercept = @trace(normal(0, 2), :intercept)
    noise = @trace(gamma(1, 1), :noise)
    
    # THIS PART IS NEW: it allows a more robust inference scheme by modeling outliers.
    # prob_outlier is the proportion of the dataset that don't fit a linear relationship (outliers) 
    prob_outlier = @trace(uniform(0, 1), :prob_outlier)
    
    # Next, we generate the actual y coordinates.
    n = length(xs)
    ys = Vector{Float64}(undef, n)
    
    for i = 1:n
        # Decide whether this point is an outlier, and set
        # mean and standard deviation accordingly
        if @trace(bernoulli(prob_outlier), :data => i => :is_outlier)
            (mu, std) = (0., 10.)
        else
            (mu, std) = (xs[i] * slope + intercept, noise)
        end
        # Sample a y value for this point
        ys[i] = @trace(normal(mu, std), :data => i => :y)
    end
    ys
end;

This model does what we want: it samples several parameters of the data-generating process, then generates data accordingly.

Let's look at the causal diagram:

<img src="./images/line_3.png" alt="Causal diagram: Line with outliers" width="600"/>

## 2. What our model is doing: visualizing the prior 

Let's visualize what our model is doing by drawing some samples from the prior. \
We use the GenViz library here which we will not focus on in this workshop.

In [3]:
using GenViz

function serialize_trace(trace)
    assmt = Gen.get_choices(trace)
    (xs,) = Gen.get_args(trace)
    Dict("slope" => assmt[:slope],
        "intercept" => assmt[:intercept],
        "inlier_std" => assmt[:noise],
        "y-coords" => [assmt[:data => i => :y] for i in 1:length(xs)],
        "outliers" => [assmt[:data => i => :is_outlier] for i in 1:length(xs)])
end;

server = VizServer(8091);

Finally, we generate some data and draw it:

In [4]:
# Get some x coordinates and initialize a visualization
xs = collect(range(-5, stop=5, length=20))
viz = Viz(server, joinpath(@__DIR__, "../tutorials/regression-viz/dist"), [xs])

# Generate ten traces and draw them into the visualization
for i=1:10
    (trace, _) = Gen.generate(model, (xs,))
    ys = Gen.get_retval(trace)
    putTrace!(viz, "t$(i)", serialize_trace(trace))
end

# Display the visualization in this notebook
displayInNotebook(viz)

Traces

Note that an outlier can occur anywhere — including close to the line — and that our model is capable of generating datasets in which the vast majority of points are outliers.

## 3. The problem with generic importance sampling  <a name="generic-importance"></a>

To motivate the need for more complex inference algorithms, let's begin by using the simple importance sampling method from the previous tutorial, and thinking about where it fails.

First, let us create a synthetic dataset to do inference _about_.

In [5]:
function make_synthetic_dataset(n)
    Random.seed!(1)
    prob_outlier = 0.2
    true_inlier_noise = 0.5
    true_outlier_noise = 5.0
    true_slope = -1
    true_intercept = 2
    xs = collect(range(-5, stop=5, length=n))
    ys = Float64[]
    for (i, x) in enumerate(xs)
        if rand() < prob_outlier
            y = randn() * true_outlier_noise
        else
            y = true_slope * x + true_intercept + randn() * true_inlier_noise
        end
        push!(ys, y)
    end
    (xs, ys)
end
    
(xs, ys) = make_synthetic_dataset(20);

In Gen, we express our _observations_ as an _Assignment_ that constrains the values of certain random choices to equal their observed values. Let's write a helper for that:

In [6]:
function make_constraints(ys::Vector{Float64})
    constraints = Gen.choicemap()
    for i=1:length(ys)
        constraints[:data => i => :y] = ys[i]
    end
    constraints
end;

We can apply it to our dataset's vector of `ys` to make a set of constraints for doing inference:

In [7]:
observations = make_constraints(ys);

Now, we use the library function `importance_resampling` to draw approximate posterior samples given those observations:

In [8]:
function logmeanexp(scores)
    logsumexp(scores) - log(length(scores))
end;

In [9]:
viz = Viz(server, joinpath(@__DIR__, "../tutorials/regression-viz/dist"), [xs])
log_probs = Vector{Float64}(undef, 10)
for i=1:10
    (tr, _) = Gen.importance_resampling(model, (xs,), observations, 2000)
    putTrace!(viz, "t$(i)", serialize_trace(tr))
    log_probs[i] = Gen.get_score(tr)
end
displayInNotebook(viz)
println("Average log probability: $(logmeanexp(log_probs))")

Traces

Average log probability: -63.159134677588995


We see here that importance sampling hasn't completely failed: it generally finds a reasonable position for the line. But the details are off: there is little logic to the outlier classification, and the inferred noise around the line is too wide. The problem is that there are just too many variables to get right, and so sampling everything in one go is highly unlikely to produce a perfect hit.

In the remainder of this notebook, we'll explore techniques for finding the right solution _iteratively_, beginning with an initial guess and making many small changes, until we achieve a reasonable posterior sample.

## 4. MCMC Inference  

### 4.1 What is MCMC? <a name="mcmc-1"></a>

So far, we've only been sampling traces that were indepent of each other. Let's now iteratively build chains of such traces where an iteration brings us closer to a good explanation of observations. 

The general shape of an MCMC algorithm is as follows: 
- We begin by **sampling an intial setting of all unobserved variables**; in Gen, we produce an initial _trace_ consistent with (but not necessarily _probable_ given) our observations. 
- Then, in a long-running loop, **we make small, stochastic changes to the trace**; in order for the algorithm to be asymptotically correct, these stochastic updates must satisfy certain probabilistic properties. 

One common way of ensuring that the updates do satisfy those properties is to compute a  **Metropolis-Hastings acceptance ratio**. Essentially, after proposing a change to a trace, we add an "accept or reject" step that stochastically decides whether to commit the update or to revert it.

Gen's `metropolis_hastings` function _automatically_ adds this "accept/reject" check (including the correct computation of the probability of acceptance or rejection).

### 4. 2 Block Resimulation <a name="mcmc-2"></a>

One of the simplest strategies we can use is called Resimulation MH, and it works as follows.

We begin, as in most iterative inference algorithms, by sampling an initial trace from our model, fixing the observed choices to their observed values.

```julia
# Gen's `initialize` function accepts a model, a tuple of arguments to the model,
# and an Assignment representing observations (or constraints to satisfy). It returns
# a complete trace consistent with the observations, and an importance weight.
(tr, _) = initialize(model, (xs,), observations)
```

Then, in each iteration of our program, we propose changes to all our model's variables in "blocks," by erasing a set of variables from our current trace and _resimulating_ them from the model. After resimulating each block of choices, we perform an accept/reject step, deciding whether the proposed changes are worth making. 

```julia
# Pseudocode
for iter=1:500
    tr = maybe_update_block_1(tr)
    tr = maybe_update_block_2(tr)
    ...
    tr = maybe_update_block_n(tr)
end
```

For the regression problem, here is one possible blocking of choices:

**Block 1: `slope`, `intercept`, and `noise`.** These parameters determine the linear relationship; resimulating them is like picking a new line. We know from our importance sampling experiment above that before too long, we're bound to sample something close to the right line.

**Blocks 2 through N+1: Each `is_outlier`, in its own block.** One problem we saw with importance sampling in this problem was that it tried to sample _every_ outlier classification at once, when in reality the chances of a single sample that correctly classifies all the points are very low. Here, we can choose to resimulate each `is_outlier` choice separately, and for each one, decide whether to use the resimulated value or not.

**Block N+2: `prob_outlier`.** Finally, we can propose a new `prob_outlier` value; in general, we can expect to accept the proposal when it is line with the current hypothesized proportion of `is_outlier` choices that are set to `true`.

In [10]:
# Perform a single block resimulation update of a trace.
function block_resimulation_update(tr)
    # Block 1: Update the line's parameters
    line_params = select(:noise, :slope, :intercept)
    (tr, _) = mh(tr, line_params)
    
    # Blocks 2-N+1: Update the outlier classifications
    (xs,) = get_args(tr)
    n = length(xs)
    for i=1:n
        (tr, _) = mh(tr, select(:data => i => :is_outlier))
    end
    
    # Block N+2: Update the prob_outlier parameter
    (tr, _) = mh(tr, select(:prob_outlier))
    
    # Return the updated trace
    tr
end;

All that's left is to (a) obtain an initial trace, and then (b) run that update in a loop for as long as we'd like:

In [11]:
function block_resimulation_inference(xs, ys)
    observations = make_constraints(ys)
    (tr, _) = generate(model, (xs,), observations)
    for iter=1:500
        tr = block_resimulation_update(tr)
    end
    tr
end;

Let's test it out:

In [12]:
scores = Vector{Float64}(undef, 10)
for i=1:10
    @time tr = block_resimulation_inference(xs, ys)
    scores[i] = get_score(tr)
end
println("Log probability: ", logmeanexp(scores))

  3.614930 seconds (9.69 M allocations: 890.178 MiB, 8.09% gc time)
  2.147976 seconds (9.15 M allocations: 862.804 MiB, 10.82% gc time)
  1.996847 seconds (9.15 M allocations: 862.805 MiB, 12.05% gc time)
  1.900715 seconds (9.15 M allocations: 862.804 MiB, 11.61% gc time)
  2.119700 seconds (9.15 M allocations: 862.805 MiB, 10.57% gc time)
  1.702746 seconds (9.15 M allocations: 862.805 MiB, 12.15% gc time)
  1.858250 seconds (9.15 M allocations: 862.803 MiB, 10.84% gc time)
  1.956809 seconds (9.15 M allocations: 862.804 MiB, 10.95% gc time)
  1.876212 seconds (9.15 M allocations: 862.804 MiB, 11.18% gc time)
  1.694434 seconds (9.15 M allocations: 862.804 MiB, 11.46% gc time)
Log probability: -48.7430917400902


We note that this is **significantly better than importance sampling**, even if we run importance sampling for about the same amount of (wall-clock) time per sample:

In [13]:
scores = Vector{Float64}(undef, 10)
for i=1:10
    @time (tr, _) = importance_resampling(model, (xs,), observations, 17000)
    scores[i] = get_score(tr)
end
println("Log probability: ", logmeanexp(scores))

  3.218261 seconds (12.21 M allocations: 1.204 GiB, 11.75% gc time)
  2.558897 seconds (12.21 M allocations: 1.204 GiB, 10.97% gc time)
  3.560484 seconds (12.21 M allocations: 1.204 GiB, 10.50% gc time)
  2.845274 seconds (12.21 M allocations: 1.204 GiB, 12.43% gc time)
  2.426851 seconds (12.21 M allocations: 1.204 GiB, 12.83% gc time)
  2.433463 seconds (12.21 M allocations: 1.204 GiB, 12.06% gc time)
  2.361190 seconds (12.21 M allocations: 1.204 GiB, 11.56% gc time)
  2.484075 seconds (12.21 M allocations: 1.204 GiB, 11.51% gc time)
  2.306404 seconds (12.21 M allocations: 1.204 GiB, 13.32% gc time)
  2.411526 seconds (12.21 M allocations: 1.204 GiB, 11.99% gc time)
Log probability: -65.28199934822558


It's one thing to see a log probability increase; it's better to understand what the inference algorithm is actually doing, and to see _why_ it's doing better.

A great tool for debugging and improving MCMC algorithms is visualization. We can use GenViz's `displayInNotebook(viz) do ... end` syntax to produce an animated visualization:

In [14]:
viz = Viz(server, joinpath(@__DIR__, "../tutorials/regression-viz/dist"), [xs, ys])
Random.seed!(2)
displayInNotebook(viz) do
    (tr, _) = generate(model, (xs,), observations)
    putTrace!(viz, "t", serialize_trace(tr))
    for iter = 1:500
        tr = block_resimulation_update(tr)
        
        # Visualize and sleep for clearer animation
        putTrace!(viz, "t", serialize_trace(tr))
        sleep(0.1)
    end
end

Traces

### 4.3 Gaussian Drift MH  <a name="mcmc-3"></a>

So far, we've seen one form of incremental trace update:

```julia
(tr, did_accept) = mh(tr, select(:address1, :address2, ...))
```

This update is incremental in that it only proposes changes to part of a trace (the selected addresses). But when computing _what_ changes to propose, it ignores the current state completely and resimulates all-new values from the model.

That wholesale resimulation of values is often not the best way to search for improvements. To that end, Gen also offers a more general flavor of MH:

```julia
(tr, did_accept) = mh(tr, custom_proposal, custom_proposal_args)
```

A "custom proposal" is just what it sounds like: whereas before, we were using the _default resimulation proposal_ to come up with new values for the selected addresses, we can now pass in a generative function that samples proposed values however it wants.

For example, here is a custom proposal that takes in a current trace, and proposes a new slope and intercept by randomly perturbing the existing values:

In [15]:
@gen function line_proposal(trace)
    choices = get_choices(trace)
    slope = @trace(normal(choices[:slope], 0.5), :slope)
    intercept = @trace(normal(choices[:intercept], 0.5), :intercept)
end;

This is often called a **"Gaussian drift" proposal**, because it essentially amounts to proposing steps of a random walk. (What makes it different from a random walk is that we will still use an MH accept/reject step to make sure we don't wander into areas of very low probability.)

To use the proposal, we write:

```julia
(tr, did_accept) = mh(tr, line_proposal, ())
```

Let's swap it into our update:

In [16]:
function gaussian_drift_update(tr)
    # Block 1: Gaussian drift on line params
    (tr, _) = mh(tr, line_proposal, ())
    
    # Block 2-N+1: Update the outlier classifications
    (xs,) = get_args(tr)
    n = length(xs)
    for i=1:n
        (tr, _) = mh(tr, select(:data => i => :is_outlier))
    end
    
    # Block N+2: Update the prob_outlier parameter
    (tr, w) = mh(tr, select(:prob_outlier))
    (tr, w) = mh(tr, select(:noise))
    tr
end;


# Only FYI: This is the old inference scheme:
"""function block_resimulation_update(tr)
    # Block 1: Update the line's parameters
    line_params = select(:noise, :slope, :intercept)
    (tr, _) = mh(tr, line_params)
    
    # Blocks 2-N+1: Update the outlier classifications
    (xs,) = get_args(tr)
    n = length(xs)
    for i=1:n
        (tr, _) = mh(tr, select(:data => i => :is_outlier))
    end
    
    # Block N+2: Update the prob_outlier parameter
    (tr, _) = mh(tr, select(:prob_outlier))
    
    # Return the updated trace
    tr
end;"""

"function block_resimulation_update(tr)\n    # Block 1: Update the line's parameters\n    line_params = select(:noise, :slope, :intercept)\n    (tr, _) = mh(tr, line_params)\n    \n    # Blocks 2-N+1: Update the outlier classifications\n    (xs,) = get_args(tr)\n    n = length(xs)\n    for i=1:n\n        (tr, _) = mh(tr, select(:data => i => :is_outlier))\n    end\n    \n    # Block N+2: Update the prob_outlier parameter\n    (tr, _) = mh(tr, select(:prob_outlier))\n    \n    # Return the updated trace\n    tr\nend;"

If we compare the Gaussian Drift proposal visually with our old algorithm, we can see how the new behavior helps:

In [17]:
viz = Viz(server, joinpath(@__DIR__, "../tutorials/regression-viz/dist"), [xs, ys])

# Set random seed for a reproducible animation
Random.seed!(30)

# Create the animation
displayInNotebook(viz) do
    # Get an initial trace
    (tr1, _) = generate(model, (xs,), observations)
    tr2 = tr1
    
    # Visualize the initial trace twice
    putTrace!(viz, 1, serialize_trace(tr1))
    putTrace!(viz, 2, serialize_trace(tr2))
    sleep(1)
    
    # Improve both traces
    for iter = 1:400
        # Gaussian drift update in trace 1
        tr1 = gaussian_drift_update(tr1)
        # Block resimulation update in trace 2
        tr2 = block_resimulation_update(tr2)
        
        # Visualize and sleep for clearer animation
        putTrace!(viz, 1, serialize_trace(tr1))
        putTrace!(viz, 2, serialize_trace(tr2))
        sleep(0.02)
    end
end

Traces

<hr />

A more quantitative comparison demonstrates that our change has definitely improved our inference quality:

In [24]:
Random.seed!(1)
function gaussian_drift_inference()
    (tr, _) = generate(model, (xs,), observations)
    for iter=1:500
        tr = gaussian_drift_update(tr)
    end
    tr
end

scores = Vector{Float64}(undef, 10)
for i=1:10
    @time tr = gaussian_drift_inference()
    scores[i] = get_score(tr)
end
println("Log probability: ", logmeanexp(scores))

  1.852348 seconds (9.70 M allocations: 916.279 MiB, 16.85% gc time)
  3.198193 seconds (9.70 M allocations: 916.279 MiB, 11.89% gc time)
  3.411584 seconds (9.70 M allocations: 916.281 MiB, 11.69% gc time)
  2.930922 seconds (9.70 M allocations: 916.280 MiB, 12.73% gc time)
  2.800965 seconds (9.70 M allocations: 916.280 MiB, 11.79% gc time)
  3.462290 seconds (9.70 M allocations: 916.280 MiB, 11.03% gc time)
  3.305600 seconds (9.70 M allocations: 916.280 MiB, 11.94% gc time)
  3.623995 seconds (9.70 M allocations: 916.279 MiB, 10.79% gc time)
  3.159080 seconds (9.70 M allocations: 916.279 MiB, 11.67% gc time)
  2.056447 seconds (9.70 M allocations: 916.281 MiB, 13.12% gc time)
Log probability: -45.87018354537137


In [25]:
<hr />

LoadError: syntax: "<" is not a unary operator

### 4.4 Data-driven proposals: heuristics to guide the process  <a name="mcmc-4"></a>

In this section, we'll look at another strategy for improving MCMC inference: using arbitrary heuristics to make smarter proposals. In particular, we'll use a method called "Random Sample Consensus" (or RANSAC) to quickly find promising settings of the slope and intercept parameters.

RANSAC works as follows:
1. We repeatedly choose a small random subset of the points, say, of size 3.
2. We do least-squares linear regression to find a line of best fit for those points.
3. We count how many points (from the entire set) are near the line we found.
4. After a suitable number of iterations (say, 10), we return the line that had the highest score.

Here's our implementation of the algorithm in Julia:

In [19]:
import StatsBase

struct RANSACParams
    # the number of random subsets to try
    iters::Int

    # the number of points to use to construct a hypothesis
    subset_size::Int

    # the error threshold below which a datum is considered an inlier
    eps::Float64
    
    function RANSACParams(iters, subset_size, eps)
        if iters < 1
            error("iters < 1")
        end
        new(iters, subset_size, eps)
    end
end


function ransac(xs::Vector{Float64}, ys::Vector{Float64}, params::RANSACParams)
    best_num_inliers::Int = -1
    best_slope::Float64 = NaN
    best_intercept::Float64 = NaN
    for i=1:params.iters
        # select a random subset of points
        rand_ind = StatsBase.sample(1:length(xs), params.subset_size, replace=false)
        subset_xs = xs[rand_ind]
        subset_ys = ys[rand_ind]
        
        # estimate slope and intercept using least squares
        A = hcat(subset_xs, ones(length(subset_xs)))
        slope, intercept = A\subset_ys
        
        ypred = intercept .+ slope * xs

        # count the number of inliers for this (slope, intercept) hypothesis
        inliers = abs.(ys - ypred) .< params.eps
        num_inliers = sum(inliers)

        if num_inliers > best_num_inliers
            best_slope, best_intercept = slope, intercept
            best_num_inliers = num_inliers
        end
    end

    # return the hypothesis that resulted in the most inliers
    (best_slope, best_intercept)
end;

We can now wrap it in a Gen proposal that calls out to RANSAC, then samples a slope and intercept near the one it proposed.

In [20]:
@gen function ransac_proposal(prev_trace, xs, ys)
    (slope, intercept) = ransac(xs, ys, RANSACParams(10, 3, 1.))
    @trace(normal(slope, 0.1), :slope)
    @trace(normal(intercept, 1.0), :intercept)
end;

One iteration of our update algorithm will now look like this:

In [21]:
function ransac_update(tr)
    # Use RANSAC to (potentially) jump to a better line from wherever we are
    (tr, _) = mh(tr, ransac_proposal, (xs, ys))
    
    # Spend a while refining the parameters, using Gaussian drift
    # to tune the slope and intercept, and resimulation for the noise
    # and outliers.
    for j=1:20
        (tr, _) = mh(tr, select(:prob_outlier))
        (tr, _) = mh(tr, select(:noise))
        (tr, _) = mh(tr, line_proposal, ())
        # Reclassify outliers
        for i=1:length(get_args(tr)[1])
            (tr, _) = mh(tr, select(:data => i => :is_outlier))
        end
    end
    tr
end;

We can now run our main loop for just 5 iterations, and achieve pretty good results. (Of course, since we do 20 inner loop iterations in `ransac_update`, this is really closer to 100 iterations.) The running time is significantly lower than before, without a real dip in quality:

In [22]:
function ransac_inference()
    (slope, intercept) = ransac(xs, ys, RANSACParams(10, 3, 1.))
    slope_intercept_init = choicemap()
    slope_intercept_init[:slope] = slope
    slope_intercept_init[:intercept] = intercept
    (tr, _) = generate(model, (xs,), merge(observations, slope_intercept_init))
    for iter=1:5
        tr = ransac_update(tr)
    end
    tr
end

scores = Vector{Float64}(undef, 10)
for i=1:10
    @time tr = ransac_inference()
    scores[i] = get_score(tr)
end
println("Log probability: ", logmeanexp(scores))

  1.368805 seconds (2.25 M allocations: 206.906 MiB, 7.12% gc time)
  0.345814 seconds (1.95 M allocations: 191.984 MiB, 14.14% gc time)
  0.340267 seconds (1.95 M allocations: 191.984 MiB, 18.26% gc time)
  0.338231 seconds (1.95 M allocations: 191.984 MiB, 17.04% gc time)
  0.385939 seconds (1.95 M allocations: 191.984 MiB, 18.39% gc time)
  0.352766 seconds (1.95 M allocations: 191.984 MiB, 14.94% gc time)
  0.684405 seconds (1.95 M allocations: 191.984 MiB, 12.13% gc time)
  0.814702 seconds (1.95 M allocations: 191.984 MiB, 13.09% gc time)
  0.628896 seconds (1.95 M allocations: 191.984 MiB, 15.33% gc time)
  0.398537 seconds (1.95 M allocations: 191.983 MiB, 14.79% gc time)
Log probability: -47.2627614835285


Let's visualize the algorithm:

In [28]:
viz = Viz(server, joinpath(@__DIR__, "../tutorials/regression-viz/dist"), [xs, ys])
displayInNotebook(viz) do
    (slope, intercept) = ransac(xs, ys, RANSACParams(10, 3, 1.))
    slope_intercept_init = choicemap()
    slope_intercept_init[:slope] = slope
    slope_intercept_init[:intercept] = intercept
    (tr, _) = generate(model, (xs,), merge(observations, slope_intercept_init))
    putTrace!(viz, "t", serialize_trace(tr))
    for iter = 1:5
        (tr, _) = mh(tr, ransac_proposal, (xs, ys))
    
        # Spend a while refining the parameters, using Gaussian drift
        # to tune the slope and intercept, and resimulation for the noise
        # and outliers.
        for j=1:20
            (tr, _) = mh(tr, select(:prob_outlier))
            (tr, _) = mh(tr, select(:noise))
            (tr, _) = mh(tr, line_proposal, ())
            # Reclassify outliers
            for i=1:length(get_args(tr)[1])
                (tr, _) = mh(tr, select(:data => i => :is_outlier))
            end
            putTrace!(viz, "t", serialize_trace(tr))
            sleep(0.1)
        end
    end
end

Traces